In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso)
from sklearn.preprocessing import MinMaxScaler
#from sklearn.cross_validation import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("/kaggle/input/housesalesprediction/kc_house_data.csv")

In [ ]:
df.head()

In [ ]:
df.shape
#so there are 21 columns and 21613 rows


In [ ]:
df.info()

It looks like we don't have any missing values and we have some datatypes which as to be converted

In [ ]:
df.date=pd.to_datetime(df.date)
df.date.head()

In [ ]:
df.describe()

In [ ]:
#to drop any duplicates if any exist
df.drop_duplicates(keep='first',inplace=True)
help(df.drop_duplicates)

In [ ]:
df.shape

We need to drop few columns which are not necessary for our model

In [ ]:
#if there is any na values drop them there are no na values as seen before
df.dropna(axis=0,inplace=True)
df.shape

In [ ]:
#lets have a look at our variables 
df.columns

In [ ]:
df.drop(['id','zipcode','lat','long'],inplace=True,axis=1)

In [ ]:
import seaborn as sns
sns.pairplot(df)

**As we can see that pairplot gives relationship between all the variables. along the diagonal we see histogram and other plots show us scatter plot of the variables **

In [ ]:
#to visualise the correlation heatmap is the best method lets see it
plt.figure(figsize=(20,8))
sns.heatmap(df.corr(),annot=True) #here it is necessary to use annot=True to get the correlation values printed on map
plt.show()

- so here our target is price we see the values of that row with respect to other variables to decide which are most important features

- other rows just gives the values among themselves(which helps in deciding whether there is presence of multicollinerity)

- sqft_lot15 and sqft_lot are same correlated features so drop one
- sqft_living and sqft_living15 are correlated
- 

In [ ]:
df.corr()

In [ ]:
#lets add features which have the little high correlation values 
cols=['sqft_living15','sqft_above','sqft_living15','sqft_lot15',]
df_fil=df[cols]

In [ ]:
df_fil.head()

In [ ]:
#describe function is used to check for any outliers or to observe pattern among values and different percentile values
df_fil.describe()

In [ ]:
help(df.describe)

##### There isn't much difference between the median(50% values) and mean of the variables so it tells us that distribution is not skewed

In [ ]:
sns.pairplot(x_vars=['bathrooms','sqft_living','grade','sqft_above','sqft_living15'],y_vars='price',data=df_fil)

In [ ]:
df_fil.corr()

- it looks like sqft_living is having strong correlation with all other features so let's drop it

In [ ]:
df_fil.drop('sqft_living',axis=1,inplace=True)

In [ ]:
df_fil.columns

In [ ]:
df_fil.head()

In [ ]:
#lets divide our data into independent variables X and dependent variable 
X=df_fil.copy()
X.shape
X.head()

In [ ]:
y=df.price

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Let's divide our data into test and train parts
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=0)

In [ ]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
import statsmodels.api as sm

# Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

X_train[['bathrooms', 'grade', 'sqft_above', 'sqft_living15']] = scaler.fit_transform(X_train[['bathrooms', 'grade', 'sqft_above', 'sqft_living15']])

X_train.head()

In [ ]:
X_train_sm=sm.add_constant(X_train)
lr=sm.OLS(y_train,X_train_sm)
lr=lr.fit()
lr.params
lr.summary()

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(X_train,y_train)

In [ ]:
y_train_pred=lr.predict(X_train)

In [ ]:
r2_score(y_train,y_train_pred)

In [ ]:
y_test_pred=lr.predict(X_test)
y_test_pred.head()

In [ ]:
help(r2_score)

In [ ]:
lr_new.Summary()

In [ ]:
X=df_fil['bathrooms','grade','sqft_above','sqft_living15'].copy()

In [ ]:
print(X.columns)

In [ ]:
print(X.head(),y.head())

In [ ]:
#this function gives us the correlation values
df.corr()

In [ ]:
# Inspecting type
print(df.dtypes)

In [ ]:
df.columns


In [ ]:
df=df.drop(['date', 'id'], axis=1)

In [ ]:
df.columns
#to see the columns dropped

In [ ]:
with sns.plotting_context("notebook",font_scale=2.5):
    g = sns.pairplot(df[['sqft_lot','sqft_above','price','sqft_living','bedrooms']], 
                 hue='bedrooms', palette='tab20',size=6)
g.set(xticklabels=[]);

In [ ]:
str_list = [] # empty list to contain columns with strings (words)
for colname, colvalue in df.iteritems():
    if type(colvalue[1]) == str:
         str_list.append(colname)
# Get to the numeric columns by inversion            
num_list = df.columns.difference(str_list) 
# Create Dataframe containing only numerical features
df_num = df[num_list]
f, ax = plt.subplots(figsize=(16, 12))
plt.title('Pearson Correlation of features')
# Draw the heatmap using seaborn
#sns.heatmap(house_num.astype(float).corr(),linewidths=0.25,vmax=1.0, square=True, cmap="PuBuGn", linecolor='k', annot=True)
sns.heatmap(df_num.astype(float).corr(),linewidths=0.25,vmax=1.0, square=True, cmap="cubehelix", linecolor='k', annot=True)

In [ ]:
from sklearn.model_selection import train_test_split
X = df.iloc[:,1:].values
y = df.iloc[:,0].values
#splitting dataset into training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 0)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predicting the Test set results
y_pred = regressor.predict(X_test)

In [ ]:
print(y_pred)